In [ ]:
import pandas as pd

import graphviz

from sklearn.model_selection import train_test_split, KFold, cross_val_score, RepeatedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn import tree
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
models = {
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
}

In [ ]:
dataset = pd.read_csv('../../data/dataset_old_features.csv')
df_champions = pd.read_csv('dataset-champions.csv')
X = dataset.drop('game', axis=1)
X = X.drop('result', axis=1)
y = dataset['result']

In [ ]:
new_X = X

for index, row in new_X.iterrows():
    new_X.loc[index, 'bluetop'] = int(df_champions.loc[df_champions['id'] == row['bluetop']]['key'])    
    new_X.loc[index, 'bluejungle'] = int(df_champions.loc[df_champions['id'] == row['bluejungle']]['key'])
    new_X.loc[index, 'bluemid'] = int(df_champions.loc[df_champions['id'] == row['bluemid']]['key'])    
    new_X.loc[index, 'blueadc'] = int(df_champions.loc[df_champions['id'] == row['blueadc']]['key'])
    new_X.loc[index, 'bluesupport'] = int(df_champions.loc[df_champions['id'] == row['bluesupport']]['key'])

    new_X.loc[index, 'redtop'] = int(df_champions.loc[df_champions['id'] == row['redtop']]['key'])
    new_X.loc[index, 'redjungle'] = int(df_champions.loc[df_champions['id'] == row['redjungle']]['key'])
    new_X.loc[index, 'redmid'] = int(df_champions.loc[df_champions['id'] == row['redmid']]['key'])
    new_X.loc[index, 'redadc'] = int(df_champions.loc[df_champions['id'] == row['redadc']]['key'])
    new_X.loc[index, 'redsupport'] = int(df_champions.loc[df_champions['id'] == row['redsupport']]['key'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(new_X, y, train_size=0.8)
kfold10 = RepeatedKFold(n_splits=10,n_repeats=5)
for name, model in models.items():
    scoresCV10 = cross_val_score(model, new_X, y, cv=kfold10)
    print('RESULTADOS DO MODELO {}:'.format(name))
    print(scoresCV10)

In [ ]:
model = models['Decision Tree'].fit(new_X,y)
dot_data = tree.export_graphviz(models['Decision Tree'], out_file=None,
                                filled=True)

# Draw graph
graph = graphviz.Source(dot_data, format="png")
filename = graph.render('outputs/decision-tree',view=True)

In [ ]:
model = models['Random Forest'].fit(new_X,y)

# Extract single tree
estimator = model.estimators_[5]

# Export as dot file
export_graphviz(estimator, out_file='outputs/random-forest.dot', 
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)

call(['dot', '-Tpng', 'outputs/random-forest.dot', '-o', 'outputs/random-forest.png', '-Gdpi=600'])